# Сопоставление фотографий Прокудина-Горского

Наша задача будет состоять в том, чтобы совместить 3 канала цветов с фотоплёнок С.М.Прокудина-Горского, дошедших до нас с начала XX века.
Вот как они выглядят:

![Photo](img_black_white.png)

У нас есть три канала:
- сначала синий
- потом зеленый
- нижний красный
Они расположены последовательно 

## Подробно разберём план решения этого задания:
1. Загружаем скан фотопленки из файла
2. Преобразуем значения пикселей изображения в вещественные
3. Делим изображение на три равные части по высоте, обрезаем края (например, по 5%)
4. Сдвигаем синий канал, накладываем на зеленый, считаем похожесть
5. Сдвигаем красный канал, накладываем на зеленый, считаем похожесть
6. Совмещаем три канала с наилучшими сдвигами

#### Разберём немного подробнее.
Сначала делаем так:
- Получили 3 картинки с обрезанными краями - по одной для каждого канала.
- Теперь берём 2 канала: синий и зелёный - и начинаем двигать синий относительно зелёного:
Т.е. переберём все возможные сдвиги синего относительно зелёного как по вертикали, так и по горизонтали от -15 до +15 пикселей - и запомним сдвиг с наибольшей похожестью.

Похожесть, это какое-то положительное число, чем больше похожесть, тем лучше результат.

### Как мы будем вычислять похожесть?
Первое, что приходит в голову - это брать всякий раз перекрывающуюся область и считать по ней похожесть каким-то образом. Но это не лучший вариант из-за того, что эта область будет каждый раз разной, это может быть сложным и сильно отразится на времени выполнения.  

Мы будем поступать по-другому: немного преобразовывать синий канал, а потом целиком накладывать на зелёный. Это будет выглядеть так:  
Мы рассмотрим операцию циклического сдвига. Она состоит в том, что мы берём некоторое количество столбцов (или строк) и передвигаем их в конец.
Таким образом у нас будет получаться полноразмерный синий канал, который мы будем целиком накладывать на зелёный.
Здесь есть проблема в том, что правая часть синего изображения будет совсем не похожа на ту же часть изображения с зелёного канала, потому что она взята с противоположной стороны. Однако это не будет вносить сильной ошибки в расчёты по причине того, что ширина этой части не превышает 15 пикселей.
В Python за такой сдвиг отвечает функция numpy.roll(), имеющая 3 аргумента.

In [ ]:
from numpy import roll
help(roll)

- Первый аргумент - это сдвигаемое изображение (у нас канал B),
-  2-й аргумент - это величина сдвига по горизонтали либо по вертикали, и
-  3-й аргумент - это число axis, задающее ось, по которой мы сдвигаем (если axis=0 - то сдвигаем строки, если axis=1 - то сдвигаем столбцы).

### Теперь разберёмся с функцией похожести.
Для этого будем считать корреляцию: поэлементное умножение пикселей двух каналов и последующее их суммирование.
### Как проверить качество решения?
Автоматическая проверка будет производиться на сайте. Что для неё является разметкой?
Разметка - это 3 точки (по одной для каждого канала), которые при совмещении каналов должны либо совместиться в одну, либо находиться друг к другу очень близко. Таким образом, именно это и будет требоваться при автоматической проверке.

### Немного подробнее.
Нам нужно будет реализовать функцию align:
На вход эта функция принимает изображение, а также точку - строку и столбец для зелёной точки. Функция должна произвести все вышеописанные вычисления и в качестве результата выдать координаты для синей и зелёной точек.

Таким образом, имея такую функцию и точки разметки, можно легко оценить качество работы программы. Для этого можно посчитать величину diff как сумму абсолютных отклонений для всех 4-х координат:
После этого можно сравнить diff c некоторым числом (например, 5 пикселей) и сделать вывод о том, хорошо ли работает наша программа.